In [ ]:
import sys
sys.path.append("../..")

In [ ]:
from src.marks_reader import *
from src.competition_data import *
from src.optimizer import *
from src.prediction_saver import *
from src.utils import *
from data.events_data import events
from datetime import datetime

In [ ]:
event_folder = 'data/classified'
only = [
    "atl_100m",
    "atl_200m",   
    "atl_400m",
    "atl_800m",
    "atl_1500m",
    "atl_5000m",
    "atl_10000m",
    "atl_100v",
    "atl_110v",
    "atl_400v",
    "atl_3000m", 
    "atl_jab",
    "atl_alt",
    "atl_per",
    "atl_lar",
    "atl_tri",
    "atl_bal",
    "atl_dis",
    "atl_mar",
    "atl_hep",
    "atl_dec",
    "atl_20km",
    "atl_50km",
    "atl_mat",
]

competition = CompetitionData(
    name="World Championships in Athletics Doha 2019", 
    events_data=events,
    start_date=datetime(2019, 9, 27),
)

data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

results = load_json('data/results.json')

In [ ]:
optimize_params(
    events=only,
    data=data,
    results=results, 
    competition=competition, 
    params=['bandwidth', 'alpha', 'sim_times', 'pond_times'],
    runcount=100,
    error_calculator=calculate_error4,
    file="opt_results4_new.json"
)